# Определение стоимости автомобилей

Сервис по продаже автомобилей разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Нужно построить модель для определения стоимости автомобилей. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.datasets import load_diabetes

In [2]:
data = pd.read_csv('datasets/autos.csv')

In [3]:
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016-03-21 09:50:58,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21 00:00:00,0,2694,2016-03-21 10:42:49
354365,2016-03-14 17:48:27,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
354366,2016-03-05 19:56:21,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
354367,2016-03-19 18:57:12,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26


In [4]:
# Функция возвращает имя датафрейма
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [5]:
# Функция возвращает информацию по каждому датафрейму из списка.
def pre_check(df_list):
    for df in df_list:
        print('=================================')
        print(get_df_name(df))
        print('_________________________________')
        print(df.info())
        print('Количество дубликатов: ', df.duplicated().sum())
        print('_________________________________')

In [6]:
# Функция удаляет те строки датафрейма, где в определённых столбцах есть значения NaN
def drop_nan_rows(df: pd.DataFrame, columns: list = []):
    print('Количество удалённых строк: ', df.columns.isna().sum())
    df.dropna(subset = columns, inplace=True)
    return df

In [7]:
pre_check([data])

_
_________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-nu

Удалим дубликаты:

In [8]:
data = data.drop_duplicates().reset_index(drop=True)

In [9]:
# Функция приводит названия колонок датафрейма к нижнему регистру
def lower_case(df: pd.DataFrame):
    df.columns = df.columns.str.lower()
    return df

In [10]:
data = lower_case(data)

Удалим строки, в которых целевой признак `NaN`, если такие есть.

In [11]:
data = drop_nan_rows(data, ['price'])

Количество удалённых строк:  0


Признаки:

- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- NotRepaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя

Целевой признак:
- Price — цена (евро)

In [12]:
# Функция возвращает Series с годом
def create_year_column(df, column, format):
    unix_time = pd.to_datetime(df[column], format=format)
    year_column = unix_time.dt.year
    return(year_column)

In [13]:
# Функция возвращает Series с месяцем
def create_month_column(df, column, format):
    unix_time = pd.to_datetime(df[column], format=format)
    month_column = unix_time.dt.month
    return(month_column)

In [14]:
# Функция возвращает Series с днём
def create_day_column(df, column, format):
    unix_time = pd.to_datetime(df[column], format=format)
    day_column = unix_time.dt.day
    return(day_column)

In [15]:
# Функция возвращает Series с днём недели
def create_weekday_column(df, column, format):
    unix_time = pd.to_datetime(df[column], format=format)
    weekday_column = unix_time.dt.weekday
    return(weekday_column)

In [16]:
# Функция, которая вставляет в датафрейм колонки с годом, месяцем, числом месяца и днём недели 

def add_ymdwd_columns(df, prefix, datetime_str_columnname, format):
    """
    df - датафрейм, в который нужно вставить колонки
    prefix - префикс для имени новой колонки
    datetime_str_columnname - имя колонки, где лежит дата-время в строковом формате
    format - строка с форматом колонки datetime_str_columnname        
    """
    year_name = prefix + "year"
    df[year_name] = create_year_column(df, datetime_str_columnname, format)
    #month_name = prefix + "month"
    #df[month_name] = create_month_column(df, datetime_str_columnname, format)
    #day_name = prefix + "day"
    #df[day_name] = create_day_column(df, datetime_str_columnname, format)
    #weekday_name = prefix + "weekday"
    #df[weekday_name] = create_weekday_column(df, datetime_str_columnname, format)

In [17]:
# Функция удаляет выбранные столбцы из датафрейма
def drop_columns(df: pd.DataFrame, columns: list):
    df = df.drop(columns, axis=1)
    return df

In [18]:
DATE_FORMAT='%Y-%m-%d'

Вставим в `data` колонку с годом создания анкеты.

In [19]:
add_ymdwd_columns(data, \
                    'reg_', \
                    'datecreated', \
                    DATE_FORMAT)

Вычислим возраст автомобиля.

In [20]:
data['carage'] = data.reg_year - data.registrationyear

Основываясь на значениях в поле `postalcode`, создадим поле `region`. Так как цена на авто может отличаться в зависимости от региона, а не от номера отделения, то возьмём самые крупные субъекты страны, то есть первый символ от индекса.

In [21]:
# Функция возвращает первые n символов слева
def left_symbols(row: str, n: int = 1):
    return row[:n]

# Проверка
left_symbols('python')

'p'

In [22]:
data.postalcode = data.postalcode.astype('str')

In [23]:
data['region'] = data.postalcode.apply(left_symbols)

In [24]:
data.head()

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,notrepaired,datecreated,numberofpictures,postalcode,lastseen,reg_year,carage,region
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57,2016,23,7
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50,2016,5,6
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46,2016,12,9
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17,2016,15,9
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21,2016,8,6


Проанализируем признаки, которые влияют или не влияют на цену автомобиля, и определим, нужно ли оставить или удалить тот или иной признак.
- *datecrawled — дата скачивания анкеты из базы.* **Не влияет. Удаляем.**
- *vehicletype — тип автомобильного кузова.* **Влияет. Оставляем.**
- *registrationyear — год регистрации автомобиля.* **Влияет. Удаляем.** Мы удаляем его из признаков, т.к. вычислили возраст автомобиля.
- *gearbox — тип коробки передач.* **Влияет. Оставляем.**
- *power — мощность (л. с.).* **Влияет. Оставляем.**
- *model — модель автомобиля.* **Влияет. Оставляем.**
- *kilometer — пробег (км).* **Влияет. Оставляем.**
- *registrationmonth — месяц регистрации автомобиля.* **Не влияет. Удаляем.** Как известно, на цену влияет год выпуска автомобиля, а месяцем обычно принебрегают.
- *fueltype — тип топлива.* **Влияет. Оставляем.**
- *brand — марка автомобиля.* **Влияет. Оставляем.**
- *notrepaired — была машина в ремонте или нет.* **Влияет. Оставляем.**
- *datecreated — дата создания анкеты.* **Не влияет. Удаляем.** По этой колонке мы нашли год публикации `reg_year`, и вычислили возраст автомобиля на момент публикации. 
- *numberofpictures — количество фотографий автомобиля*. * **Не влияет. Удаляем.** Признак скорее влияет на привлекательность для просмотра объявления, а не на рыночную стоимость.
- *postalcode — почтовый индекс владельца анкеты (пользователя).* **Влияет. Удаляем.** Так как цена на авто может отличаться в зависимости от региона, то возьмём самыфе крупные субъекты страны, то есть только первый символ от индекса.
- *lastseen — дата последней активности пользователя.* **Не влияет. Удаляем.**
- *reg_year — год публикации.* **Влияет. Удаляем.** Мы удаляем его из признаков, т.к. вычислили возраст автомобиля.
- *carage — возраст автомобиля.* **Влияет. Оставляем.**
- *region — регион объявления.* **Влияет. Оставляем.**

Преобразуем датасет, оставив только важные признаки, и заменим пропущенные значения на `unknown`.

In [25]:
data = drop_columns(data, ['datecrawled',
                           'registrationyear',
                           'registrationmonth',
                           'datecreated',
                           'numberofpictures',
                           'postalcode',
                           'lastseen',
                           'reg_year'])
data = data.fillna('unknown')
data.head()

,price,vehicletype,gearbox,power,model,kilometer,fueltype,brand,notrepaired,carage,region
0,480,unknown,manual,0,golf,150000,petrol,volkswagen,unknown,23,7
1,18300,coupe,manual,190,unknown,125000,gasoline,audi,yes,5,6
2,9800,suv,auto,163,grand,125000,gasoline,jeep,unknown,12,9
3,1500,small,manual,75,golf,150000,petrol,volkswagen,no,15,9
4,3600,small,manual,69,fabia,90000,gasoline,skoda,no,8,6


Теперь преобразуем категориальные признаки в численные методом One-Hot Encoding.

In [26]:
data_ohe = pd.get_dummies(data)

In [27]:
data_ohe

,price,power,kilometer,carage,vehicletype_bus,vehicletype_convertible,vehicletype_coupe,vehicletype_other,vehicletype_sedan,vehicletype_small,...,notrepaired_yes,region_1,region_2,region_3,region_4,region_5,region_6,region_7,region_8,region_9
0,480,0,150000,23,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,18300,190,125000,5,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0
2,9800,163,125000,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1500,75,150000,15,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,3600,69,90000,8,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354360,0,0,150000,11,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
354361,2200,0,20000,11,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
354362,1199,101,125000,16,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
354363,9200,102,150000,20,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [28]:
features = drop_columns(data_ohe, 'price')
target = data_ohe['price']

## Обучение моделей

In [29]:
features_train, features_test, target_train, target_test = \
train_test_split(features, target, test_size=0.2, random_state=12345)

In [30]:
def RMSE(y_true,y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    #print 'MSE: %2.3f' % mse
    return rmse

In [31]:
# Функция возвращает предсказания лучшей модели и target_valid
def best_model(features_train, 
                  target_train,
                  model,
                  param_grid, 
                  cv, 
                  refit):
    
    my_scorer = {'RMSE': make_scorer(RMSE, greater_is_better=False)
                #,'MAE': make_scorer(mean_absolute_error)
                }
    
    model = model
    grid = GridSearchCV(model, 
                        param_grid, 
                        cv=cv, 
                        refit=refit, 
                        scoring=my_scorer) 
    grid_result = grid.fit(features_train, target_train)
    best_model = grid_result.best_estimator_
    best_score = grid_result.best_score_
    best_params = grid.best_params_
    print(best_params)
    #print(refit, ": {:.2f}".format(best_score), "%")
    print(refit,":", best_score)
    return(best_model, best_score)

Сначала выберем лучшие модели для линейной регрессии, решающего дерева и случайного леса.

In [32]:
# Сетка параметров для линейной регрессии
PARAM_GRID_LR = {'fit_intercept': [False, True],
              'normalize': [False, True],
             'n_jobs': [-1, None]}  
PARAM_GRID_LR;

In [33]:
%%time
model_LR, RMSE_LR = best_model(features_train, 
                  target_train,
                  LinearRegression(),
                  PARAM_GRID_LR, 
                  5, 
                  'RMSE')

{'fit_intercept': True, 'n_jobs': -1, 'normalize': False}
RMSE : -3168.61638299894
Wall time: 5min 8s


In [34]:
# Сетка параметров для DecisionTreeRegressor()
PARAM_GRID_DTR = {'splitter': ['best', 'random'],
                  'max_depth': range(1,10),
                  'min_samples_leaf': range(1,5),
                 'random_state': [12345]}  
PARAM_GRID_DTR;

In [35]:
%%time
model_DTR, RMSE_DTR = best_model(features_train, 
                  target_train,
                  DecisionTreeRegressor(),
                  PARAM_GRID_DTR, 
                  5, 
                  'RMSE')

{'max_depth': 9, 'min_samples_leaf': 4, 'random_state': 12345, 'splitter': 'best'}
RMSE : -2175.347923310198
Wall time: 30min 55s


In [36]:
# Сетка параметров для RandomForestRegressor()
PARAM_GRID_RFR = {'n_estimators': [3,  10,  31, 100], #np.logspace(0, 2, 5, endpoint=True, dtype = 'int')
                  'max_depth': range(1,5),
                  #'min_samples_leaf': range(1,5),
                 'random_state': [12345]}  
PARAM_GRID_RFR;

In [37]:
%%time
model_RFR, RMSE_RFR = best_model(features_train, 
                  target_train,
                  RandomForestRegressor(),
                  PARAM_GRID_RFR, 
                  5, 
                  'RMSE')

{'max_depth': 4, 'n_estimators': 31, 'random_state': 12345}
RMSE : -2651.462474329158
Wall time: 1h 25s


Теперь попробуем градиентный бустинг.

In [66]:
# Сетка параметров для HistGradientBoostingRegressor()
PARAM_GRID_HGBR = {'learning_rate': [0.5], #[0.2 ,0.5, 0.7]
                  #'max_iter': [   1,   10,  100, 200], #np.logspace(0, 2, 5, endpoint=True, dtype = 'int'),
                  #'max_depth': range(1,5),
                  #'random_state': [12345],
                  #'verbose': [1]
                  }  
PARAM_GRID_HGBR;

In [67]:
%%time
model_HGBR, RMSE_HGBR = best_model(features_train, 
                  target_train,
                  HistGradientBoostingRegressor(),
                  PARAM_GRID_HGBR, 
                  5, 
                  'RMSE')

{'learning_rate': 0.5}
RMSE : -1787.6838305341262
Wall time: 4min 2s


## Анализ моделей

Для HistGradientBoostingRegressor RMSE меньше всего, и модель обучается значительно быстрее, чем другие:

In [50]:
abs(RMSE_LR), abs(RMSE_DTR), abs(RMSE_RFR), abs(RMSE_HGBR)

(3168.61638299894, 2175.347923310198, 2651.462474329158, 1782.9965002031531)

- Модель решающего дерева показала менее точный результат (2175), чем градиентный бустинг, но обучалась значительно дольше: 30 минут.
- Модель случайного леса показала ещё более худший результат (2651), и обучалась 1 час 30 минут.
- Модель линейной регрессии показала самый худший результат (3168), зато обучилвсь быстро: 5 минут.

Проверим модель градиентного бустинга на тестовой выборке:

In [40]:
predictions = model_HGBR.predict(features_test)
RMSE(target_test, predictions)

1778.4473925508867

Модель показала на тестовой выборке практически такой же результат, как на обучающей выборке.